In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# returns an array of the movie_id for all movies user has rated
def get_user_movies(user_id: int):
    return set([int(x) for x in ratings_df[ratings_df['user_id'] == user_id].index])
print(get_user_movies(4))

In [ ]:
def get_common_movies(user_id1: int, user_id2: int):
    user1_movies = set(get_user_movies(user_id1))
    user2_movies = set(get_user_movies(user_id2))
    return user1_movies.intersection(user2_movies)

# get_common_movies(1, 5)

In [ ]:
def get_next_avail_user_id(df:pd.DataFrame):
    return df['user_id'].max()

# get_next_avail_user_id(ratings_df)

In [ ]:
def get_user_movie_rating(user_id:int,movie_id:int):
    return int(ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df.index == movie_id)]['rating'])

# print(get_user_movie_rating(4, 34))

In [ ]:
# get highest rated movie from user2 ratings with no rating for user1
def get_movie_recommendation(user1_id:int, user2_id:int) -> int:
    common_movies = get_common_movies(user1_id, user2_id)
    movies_no_user1_rating = get_user_movies(user2_id) - common_movies
    movie_id_rating_pairs = [(movie_id, get_user_movie_rating(user2_id, movie_id)) for movie_id in movies_no_user1_rating]
    # sort by ratings
    movie_id_rating_pairs.sort(key=lambda pair: pair[1], reverse=True)
    return movie_id_rating_pairs

print(get_movie_title(get_movie_recommendation(71569, 7).pop()[0]))

In [ ]:
def get_most_similar_user(user_id: int):
    comparisons = 0
    most_similar_user = None
    most_similar_score = float(0)
    %time
    for id in ratings_df['user_id'].unique():
        similarity = get_similarity_score(user_id, id)
        if similarity != None and user_id != id:
            if similarity > most_similar_score:
                if similarity == float(1) and len(get_common_movies(user_id, id)) < 4:
                    continue
                most_similar_user = id
                most_similar_score = similarity
                print(f'updated most similar ({most_similar_user} with score of {most_similar_score})')
        comparisons += 1
        if comparisons >= 100:
            return most_similar_user

# get_most_similar_user(5)

In [ ]:
# return the top 30 most popular movies to use as the prompt for user ratings
output_file = 'recs-app/public/top_rated_movies30.json'

top_rated_movies.sort_values(by='Count', ascending=False, inplace=True)
top_rated_movies_names = top_rated_movies.index.values
top_rated_movies_slice = top_rated_movies.head(10)
top_rated_movies_slice.columns = top_rated_movies_slice.columns.get_level_values(0)
print(top_rated_movies)

top_rated_movies_slice.to_json(output_file, orient='index')


In [ ]:
def get_naive_recommendation(user_id: int):
    user2_id = get_most_similar_user(user_id)
    recs = get_movie_recommendation(user_id, user2_id)
    for rec in recs:
        print(get_movie_title(rec[0]))

# get_naive_recommendation(6)

In [ ]:
# add ratings to the dataset to make predictions
rating_pairs = ((1884, 5),(4006,1),(48516,5),(608,5),(45672,2))
def append_new_user_with_rating_pairs(r_pairs):
    new_user_id = get_next_avail_user_id(ratings_df)
    ratings = [p[1] for p in r_pairs]
    movie_ids = [p[0] for p in r_pairs]
    user_ids = [new_user_id for _ in range(0,len(r_pairs))]
    timestamps = [None for _ in range(0,len(r_pairs))]
    new_data = {
        'user_id': user_ids,
        'movie_id': movie_ids,
        'rating': ratings,
        'timestamp': timestamps
    }

    new_ratings_df = pd.DataFrame(new_data)
    appended_ratings_df = pd.concat([ratings_df, new_ratings_df])
    print(appended_ratings_df)
    # print(new_ratings_df)
    # print(ratings_df)

append_new_user_with_rating_pairs(rating_pairs)
get_naive_recommendation(71567)

In [ ]:
from scipy import stats

def get_movie_rating_count_percentile(movie_id:int):
    movie_name = get_movie_title(movie_id)
    movie_num_ratings = grouped_df[grouped_df.index == movie_name]['Count']
    percentile = stats.percentileofscore(grouped_df['Count'], movie_num_ratings)
    
    print(f'{movie_name} has been rated {int(movie_num_ratings)} times, placing it in the {percentile[0]}th percentile')
    return percentile

# print(grouped_df.index)
get_movie_rating_count_percentile(33750)
get_movie_rating_count_percentile(33639)
get_movie_rating_count_percentile(1884)
get_movie_rating_count_percentile(33621)
get_movie_rating_count_percentile(33615)
get_movie_rating_count_percentile(296)


In [ ]:
from adjustText import adjust_text


def plot_users_ratings(user1_id: int, user2_id: int):
    common_movies = get_common_movies(user1_id, user2_id)
    user_1_ratings = [get_user_movie_rating(
        user1_id, m) for m in common_movies]
    user_2_ratings = [get_user_movie_rating(
        user2_id, m) for m in common_movies]
    movie_titles = [get_movie_title(movie_id) for movie_id in common_movies]
    new_data = {
        'titles': movie_titles,
        'user1_ratings': user_1_ratings,
        'user2_ratings': user_2_ratings,
    }
    users_common_ratings_df = pd.DataFrame(data=new_data)
    ax = users_common_ratings_df.plot.scatter(
        x='user1_ratings', y='user2_ratings')

    texts = []

    for i, title in enumerate(movie_titles):
        texts.append(ax.text(user_1_ratings[i], user_2_ratings[i], title))

    adjust_text(texts)
    ax.set_xlim(0.25, 5.25)
    ax.set_ylim(0.25, 5.25)
    ax.plot()


plot_users_ratings(5, 19)


In [4]:
from movie_recommender import MovieRecommender


movies_file = 'data/ml-10M100K/movies.dat'
ratings_file = 'data/ml-10M100K/ratings.dat'
movie_rec = MovieRecommender(movies_filename=movies_file, ratings_filename=ratings_file)

Error occured while instantiating datafarmes: [Errno 2] No such file or directory: 'data/ml-10M100K/movies.dat'


AttributeError: 'MovieRecommender' object has no attribute 'movies_df'

In [ ]:
movie_rec.handle_recommendation_request(liked_movies=(1, 2, 19, 34), disliked_movies=())

In [5]:
movie_rec.top_rated_movies.sort_values(
            by='Count', ascending=False, inplace=True)
top_rated_movies_names = movie_rec.top_rated_movies.index.values
length = 500
movie_rec.top_rated_movies_slice = movie_rec.top_rated_movies.head(length)
movie_rec.top_rated_movies_slice.columns = movie_rec.top_rated_movies_slice.columns.get_level_values(0)
print(movie_rec.top_rated_movies_slice.columns)
movie_rec.top_rated_movies_slice = movie_rec.top_rated_movies_slice.drop('rating', axis=1)
# Define a lambda function to get the movie_id for a given movie name
get_movie_id = lambda movie_name: movie_rec.movies_df[movie_rec.movies_df['name'] == movie_name].index.values[0]

# Apply the lambda function to each row of the DataFrame to set the movie_id
movie_rec.top_rated_movies_slice['movie_id'] = movie_rec.top_rated_movies_slice.index.map(get_movie_id)
movie_rec.top_rated_movies_slice['title'] = movie_rec.top_rated_movies_slice.index
movie_rec.top_rated_movies_slice = movie_rec.top_rated_movies_slice.set_index('movie_id')

print(movie_rec.top_rated_movies_slice)
output_file='data/top_rated_{}.json'.format(length)

movie_rec.top_rated_movies_slice.to_json(output_file, orient='index')

NameError: name 'movie_rec' is not defined

In [ ]:
from urllib.parse import quote
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning
import requests
import bs4

def pull_movie_poster(movie_id: int) -> None:
    try:
        disable_warnings(InsecureRequestWarning)
        title = movie_rec.get_movie_title_with_year(movie_id)
        clean_title = movie_rec.get_movie_title(movie_id)
        base_url = 'https://imdb.com/find/?q='
        base_page_url = 'https://imdb.com'
        qs_title = quote(title)
        url = base_url + qs_title + '&ref_=nv_sr_sm'
        headers = requests.utils.default_headers()
        headers.update({'User-Agent': 'My User Agent 2.0',})

        print(f'search page: {url}')
        html = requests.get(url, verify=False, headers=headers).content
        soup = bs4.BeautifulSoup(html, 'lxml')
        find_title_results = soup.find_all('li', {'class': 'find-title-result'})
        for tr in find_title_results:
            div_title = tr.find_all(
                'a', {'class': 'ipc-metadata-list-summary-item__t'})
            for dt in div_title:
                # this may need a more clever way of filtering if the wrong movie posters are getting pulled
                if dt.text == clean_title or ',' in clean_title or True:
                    full_page = tr.find('a')['href']
                    full_page_html = requests.get(
                        base_page_url+full_page, verify=False, headers=headers).content
                    full_page_soup = bs4.BeautifulSoup(full_page_html, 'lxml')
                    img = full_page_soup.find('img', {'class': 'ipc-image'})
                    if img:
                        if img['src']:
                            response = requests.get(img['src'], verify=False)
                            img_content = response.content
                            with open('data/posters/'+str(movie_id)+'.jpg', 'wb') as f:
                                print(f'writing image {clean_title}')
                                f.write(img_content)
                                return
                        else:
                            print('no image source found')
                    else:
                        print('no image found')
                else:
                    print(f'mismatching title and listed name: {dt.text} != {clean_title}')
    except Exception as e:
        print(e)



In [ ]:
import pandas as pd

# movie_rec.output_to_json()
top30_input_filepath = 'data/top_rated_250.json'
top30_filepath = 'recs-app/public/top_rated_movies30.archive.json'
top30_df = pd.read_json(top30_input_filepath)
top30_df = top30_df.T
get_movie_id = lambda movie_name: movie_rec.movies_df[movie_rec.movies_df['name'] == movie_name].index.values[0]

# Apply the lambda function to each row of the DataFrame to set the movie_id
top30_df['movie_id'] = top30_df['title'].map(get_movie_id)
top30_df.to_json(top30_filepath, orient="index")
for name in top30_df['title'][10:]:
    pull_movie_poster(top30_df.loc[top30_df['title'] == name]['movie_id'].values[0])
# print(top30_df.T)


In [ ]:
g = movie_rec.get_movie_genres(1)
print(g)

In [ ]:
import numpy as np


# return the difference between average ratings for two movies
def get_movie_mean_rating_error(movie1_id: int, movie2_id: int):
    movie1_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie1_id]['rating'].values)
    movie2_ratings = np.mean(
        movie_rec.ratings_df.loc[movie_rec.ratings_df.index == movie2_id]['rating'].values)
    return abs(movie1_ratings - movie2_ratings)


# get_movie_mean_rating_error(599, 700)
# movie_rec.get_movie_users(5)
print(movie_rec.get_movie_title(1))
print(movie_rec.get_movie_title(2))
print(len(movie_rec.get_movie_users(1)))
print(len(movie_rec.get_movie_users(2)))
print(len(movie_rec.get_common_users(1, 2)))

In [ ]:
def get_movie_rating_pearson_coeff(movie1_id: int, movie2_id: int)-> float:
    common_users = movie_rec.get_common_users(movie1_id=movie1_id, movie2_id=movie2_id)
    movie1_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie1_id) for u in common_users]
    movie2_ratings = [movie_rec.get_user_movie_rating(user_id=u, movie_id=movie2_id) for u in common_users]
    coeff = np.corrcoef(np.array(movie1_ratings), np.array(movie2_ratings))[0, 1]
    movie1_title = movie_rec.get_movie_title(movie1_id)
    movie2_title = movie_rec.get_movie_title(movie2_id)
    print(f'{movie1_title} and {movie2_title} have a correlation of {"{:.2f}".format(coeff)} from {len(common_users)} users who have rated both')

get_movie_rating_pearson_coeff(1, 3)

In [ ]:
from urllib.parse import quote
import requests
import bs4

movie_id = 1
title = movie_rec.get_movie_title_with_year(movie_id)
clean_title = movie_rec.get_movie_title(movie_id)
base_url = 'https://imdb.com/find/?q='
base_page_url = 'https://imdb.com'
qs_title = quote(title)
url = base_url + qs_title + '&ref_=nv_sr_sm'
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)

print(f'search page: {url}')
html = requests.get(url, verify=False, headers=headers).content
soup = bs4.BeautifulSoup(html, 'lxml')
find_title_results = soup.find_all('li', {'class': 'find-title-result'})
for tr in find_title_results:
    div_title = tr.find_all('a', {'class': 'ipc-metadata-list-summary-item__t'})
    for dt in div_title:
        if dt.text == clean_title:
            full_page = tr.find('a')['href']
            full_page_html = requests.get(base_page_url+full_page, verify=False, headers=headers).content
            full_page_soup = bs4.BeautifulSoup(full_page_html, 'lxml')
            print(full_page_soup.prettify())
            img = full_page_soup.find('img', {'class':'ipc-image'})
            print(img['src'])
            if img['src']:
                response = requests.get(img['src'], verify=False)
                img_content = response.content
                with open('data/posters/'+str(movie_id)+'.jpg', 'wb') as f:
                    f.write(img_content)
            # img = tr.find('img')
            # if img['src']:
            #     print(img['src'])
            #     response = requests.get(img['src'], verify=False)
            #     img_content = response.content
            #     with open('data/posters/'+str(movie_id)+'.jpg', 'wb') as f:
            #         f.write(img_content)

